## Gerando arquivo para NER-NestedClinBr

Teste com formato QA.. tem como retornar as entidades descontinuas tbm?

In [31]:
import os
from pathlib import Path
import re
import pickle
import random


In [32]:
def getTiposEntidade():
    return ['Problema','Teste','Tratamento','Anatomia']

In [33]:
def replaceWhiteSpaces(str):
    return re.sub('\s{2,}',' ',str)

In [34]:
def save_obj(name, obj):
    existeDir = os.path.exists('../obj')
    if not existeDir:
        os.makedirs('../obj')
    with open('../obj/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f)

def load_obj(name):
    existeDir = os.path.exists('../obj')
    if not existeDir:
        os.makedirs('../obj')
    try:
        with open('../obj/' + name + '.pkl', 'rb') as f:
            return pickle.load(f)
    except:
        return None


In [35]:
pastasCorpus = [r'corpus\test',r'corpus\train']
#pastasCorpus = [r'corpus\test']
#pastasCorpus = [r'corpus\TESTE']

In [36]:
import re

s = 'Make the World a 2.4Better Place2.0'
pattern = r'([0-9])\.([0-9])'
replacement = r'\1y\2'
html = re.sub(pattern, replacement, s)

print(html)

Make the World a 2y4Better Place2y0


In [37]:
'''
def getDicSentences(pastaCorpus):
    #devePrintar=True
    devePrintar=False
    dic_sentences = {}
    numMaxTokensPorFrase=0 # numero tokens da maior frase
    num=0
    listaEntidades=[]
    frasesComDescontinuas=[]
    for filename in os.listdir(pastaCorpus):
        f = os.path.join(pastaCorpus, filename)
        if os.path.isfile(f):
            fileNameSemExtensao=os.path.splitext(filename)[0]
            if devePrintar:
                print('\n\n--fileName (sem extensao):--', fileNameSemExtensao)
                pass
            extension = os.path.splitext(filename)[1][1:]
            if extension=='ann':
                # tokens da frase
                fileTxt = open(os.path.join(pastaCorpus, fileNameSemExtensao)+'.txt', "r", encoding='utf-8')
                linha=fileTxt.readlines()
                fileTxt.close()
                if devePrintar:
                    print('linha:', linha)
                    pass
                frases=[]
                allFrasesString=''
                numL=0
                for l in linha:
                    numL=numL+1
                    allFrasesString = allFrasesString+l
                    if numL==1: # descarta primeira frase, que é data de criação do doc
                        #print('descartando frase:', l)
                        continue
                    if l.strip() and l.strip()!='\n':
                        #print('l:', l)
                        pattern = r'([0-9])\.([0-9])'
                        replacement = r'\1==\2'
                        novoL = re.sub(pattern, replacement, l.strip())
                        l2 = novoL.split('.') # quebrando frases
                        for l3 in l2:
                            if l3.strip() and l3.strip()!='\n':
                                novaFrase = l3.replace('\n','').replace('==','.').strip()+'.'
                                frases.append(novaFrase)
                print('frases:', frases)
                # para cada frase
                # para tokenizar nesses tokens
                print('allFrasesString:', allFrasesString)
                frasesTokens={}
                #numCaracteresTotal=42 # primeira frase ignorada
                numIndiceAnterior=0
                for frase in frases:
                    tokens=[]
                    frase2 = frase.strip().replace('/',' / ').replace(')',' ) ').replace('(',' ( ').replace(']',' ] ').replace('[',' [ ').replace(',',' , ').replace('.',' . ').replace(';',' ; ').replace('-',' - ').replace('+',' + ').replace("'"," ' ").replace(" = "," = ").replace("#"," # ").replace(" $ "," $ ").replace(" ! "," ! ").replace("?"," ? ").replace("%"," % ").replace(":"," : ").replace(">"," > ").replace("<"," < ")
                    frase2 = replaceWhiteSpaces(frase2)
                    frase2 = frase2.split()
                    for numtoken, token in enumerate(frase2):
                        if devePrintar:
                            print('token:', token)
                            print('numIndiceAnterior:', numIndiceAnterior)
                        if token!='.':
                            numCaracteresTotal = allFrasesString.find(token, numIndiceAnterior, len(allFrasesString))
                        else:
                            numCaracteresTotal=numIndiceAnterior+1
                        #tokens.append([token,numtoken])
                        tokens.append([token,numtoken, numCaracteresTotal])
                        numIndiceAnterior = numCaracteresTotal+len(token)-1
                        if numMaxTokensPorFrase<len(tokens):
                            numMaxTokensPorFrase = len(tokens)
                    frasesTokens[frase]=tokens
                    #linhaTokens=linha.copy()    
                if devePrintar:
                    print('frasesTokens:', frasesTokens)
                # agora, as entidades
                fileAnn = open(f, "r", encoding='utf-8')
                linha=fileAnn.readlines()
                fileAnn.close()

                dicEntidades={}
                for entidade_linha in linha:
                    if ';' not in entidade_linha: # tem descontinua?
                        entidade = entidade_linha.split('\t')
                        tipo_entidade = entidade[1]
                        inicio, fim = tipo_entidade.split()[1:3]
                        tipo_entidade = tipo_entidade.split()[0]
                        termos_entidade = entidade[2].replace('\n','')
                        termos_entidade = termos_entidade.strip().replace('/',' / ').replace(')',' ) ').replace('(',' ( ').replace(']',' ] ').replace('[',' [ ').replace(',',' , ').replace('.',' . ').replace(';',' ; ').replace('-',' - ').replace('+',' + ').replace("'"," ' ").replace(" = "," = ").replace("#"," # ").replace(" $ "," $ ").replace(" ! "," ! ").replace("?"," ? ").replace("%"," % ").replace(":"," : ").replace(">"," > ").replace("<"," < ")
                        dicEntidades[(int(inicio), int(fim))]=[tipo_entidade, replaceWhiteSpaces(termos_entidade)]
                    else:
                        frasesComDescontinuas.append(num)
                        #print('descontinua, linha: {}, num: {}'.format(linha, num))
                        #print('descontinua, file: {}, num: {}'.format(filename, num))
                        entidade = entidade_linha.split('\t')
                        # ex T10	Problema 244 252;279 306	dispneia aos mdoeardos-leves esforço
                        #Problema 244 252;279 306
                        entidade_temp=entidade[1].split(';')
                        entidade1=entidade_temp[0]
                        tipo_entidade = entidade1
                        inicio1, fim1 = tipo_entidade.split()[1:3]
                        tipo_entidade_string = tipo_entidade.split()[0]
                        # mandar só os termos referentes...
                        tamTermo1=int(fim1)-int(inicio1)
                        termos_entidade = entidade[2].replace('\n','')
                        termos_entidade=termos_entidade[:tamTermo1]
                        termos_entidade = termos_entidade.strip().replace('/',' / ').replace(')',' ) ').replace('(',' ( ').replace(']',' ] ').replace('[',' [ ').replace(',',' , ').replace('.',' . ').replace(';',' ; ').replace('-',' - ').replace('+',' + ').replace("'"," ' ").replace(" = "," = ").replace("#"," # ").replace(" $ "," $ ").replace(" ! "," ! ").replace("?"," ? ").replace("%"," % ").replace(":"," : ").replace(">"," > ").replace("<"," < ")
                        #print('aaaaaaaaaaaaaaaaa')
                        dicEntidades[(int(inicio1), int(fim1))]=[tipo_entidade_string, replaceWhiteSpaces(termos_entidade)]
                        #print("(int(inicio1), int(fim1)):", (int(inicio1), int(fim1)))
                        #print("tipo_entidade_string, replaceWhiteSpaces(termos_entidade):", tipo_entidade_string, replaceWhiteSpaces(termos_entidade))
                        
                        entidade2=entidade_temp[1]
                        inicio2, fim2 = entidade2.split()[0:2]
                        termos_entidade = entidade[2].replace('\n','')
                        termos_entidade=termos_entidade[tamTermo1:len(termos_entidade)]
                        termos_entidade = termos_entidade.strip().replace('/',' / ').replace(')',' ) ').replace('(',' ( ').replace(']',' ] ').replace('[',' [ ').replace(',',' , ').replace('.',' . ').replace(';',' ; ').replace('-',' - ').replace('+',' + ').replace("'"," ' ").replace(" = "," = ").replace("#"," # ").replace(" $ "," $ ").replace(" ! "," ! ").replace("?"," ? ").replace("%"," % ").replace(":"," : ").replace(">"," > ").replace("<"," < ")
                        dicEntidades[(int(inicio2), int(fim2))]=[tipo_entidade_string, replaceWhiteSpaces(termos_entidade)]
                        #print("(int(inicio2), int(fim2)):", (int(inicio2), int(fim2)))
                        #print("tipo_entidade_string, replaceWhiteSpaces(termos_entidade):", tipo_entidade_string, replaceWhiteSpaces(termos_entidade))


                #print('frasesTokens:', frasesTokens)
                indicesDic = sorted(dicEntidades.keys(), key = lambda item: item[0])
                listaIndicesJaUsados = []
                #list_students.sort(key = lambda x: x[1])   #index 1 means second element
                #print(indicesDic)
                #print('dicEntidades:', dicEntidades)
                
                for key, value in frasesTokens.items():
                    for i in indicesDic:
                        tipo_entidade,termos_entidade = dicEntidades[i]
                        ##key (i) = indices old a comparar
                        #print('key:', key)
                        #print('value:', value)
                        #print('i:', i)
                        for token in value:
                            #print('token:', token)
                            if i[0]==token[2]:
                                novo_inicio, novo_fim = [token[1],token[1]+len(termos_entidade.split())]
                                novos_indices=[]
                                for k in range(novo_inicio,novo_fim):
                                    novos_indices.append(k)
                                listaEntidades.append([termos_entidade, novos_indices, tipo_entidade])
                                #print("[termos_entidade, novos_indices, tipo_entidade]:", [termos_entidade, novos_indices, tipo_entidade])
                            else:
                                #print('else, i[0]:',i[0])
                                pass
                        
                    if len(value)>0:
                        #print('incluindo:', key)
                        dic_sentences[num]=[value, listaEntidades]
                        listaEntidades=[]
                        num=num+1 

        #print(num)
        #if num>318:
        #    break

        #if num>3:
        #    break
    print('numMaxTokensPorFrase:', numMaxTokensPorFrase)
    return dic_sentences, frasesComDescontinuas
            
'''

'\ndef getDicSentences(pastaCorpus):\n    #devePrintar=True\n    devePrintar=False\n    dic_sentences = {}\n    numMaxTokensPorFrase=0 # numero tokens da maior frase\n    num=0\n    listaEntidades=[]\n    frasesComDescontinuas=[]\n    for filename in os.listdir(pastaCorpus):\n        f = os.path.join(pastaCorpus, filename)\n        if os.path.isfile(f):\n            fileNameSemExtensao=os.path.splitext(filename)[0]\n            if devePrintar:\n                print(\'\n\n--fileName (sem extensao):--\', fileNameSemExtensao)\n                pass\n            extension = os.path.splitext(filename)[1][1:]\n            if extension==\'ann\':\n                # tokens da frase\n                fileTxt = open(os.path.join(pastaCorpus, fileNameSemExtensao)+\'.txt\', "r", encoding=\'utf-8\')\n                linha=fileTxt.readlines()\n                fileTxt.close()\n                if devePrintar:\n                    print(\'linha:\', linha)\n                    pass\n                frases

In [38]:
def getDicSentences(pastaCorpus):
    #devePrintar=True
    devePrintar=False
    dic_sentences = {}
    numMaxTokensPorFrase=0 # numero tokens da maior frase
    num=0
    listaEntidades=[]
    frasesComDescontinuas=[]
    for filename in os.listdir(pastaCorpus):
        f = os.path.join(pastaCorpus, filename)
        if os.path.isfile(f):
            fileNameSemExtensao=os.path.splitext(filename)[0]
            if devePrintar:
                print('\n\n--fileName (sem extensao):--', fileNameSemExtensao)
                pass
            extension = os.path.splitext(filename)[1][1:]
            if extension=='ann':
                # tokens da frase
                fileTxt = open(os.path.join(pastaCorpus, fileNameSemExtensao)+'.txt', "r", encoding='utf-8')
                linha=fileTxt.readlines()
                fileTxt.close()
                if devePrintar:
                    print('linha:', linha)
                    pass
                frases=[]
                allFrasesString=''
                numL=0
                for l in linha:
                    numL=numL+1
                    allFrasesString = allFrasesString+l
                    if numL==1: # descarta primeira frase, que é data de criação do doc
                        #print('descartando frase:', l)
                        continue
                    if l.strip() and l.strip()!='\n':
                        #print('l:', l)
                        pattern = r'([0-9])\.([0-9])'
                        replacement = r'\1==\2'
                        novoL = re.sub(pattern, replacement, l.strip())
                        l2 = novoL.split('.') # quebrando frases
                        for l3 in l2:
                            if l3.strip() and l3.strip()!='\n':
                                novaFrase = l3.replace('\n','').replace('==','.').strip()+'.'
                                frases.append(novaFrase)
                #print('frases:', frases)
                # para cada frase
                # para tokenizar nesses tokens
                #print('allFrasesString:', allFrasesString)
                frasesTokens={}
                #numCaracteresTotal=42 # primeira frase ignorada
                numIndiceAnterior=0
                for frase in frases:
                    tokens=[]
                    frase2 = frase.strip().replace('/',' / ').replace(')',' ) ').replace('(',' ( ').replace(']',' ] ').replace('[',' [ ').replace(',',' , ').replace('.',' . ').replace(';',' ; ').replace('-',' - ').replace('+',' + ').replace("'"," ' ").replace(" = "," = ").replace("#"," # ").replace(" $ "," $ ").replace(" ! "," ! ").replace("?"," ? ").replace("%"," % ").replace(":"," : ").replace(">"," > ").replace("<"," < ")
                    frase2 = replaceWhiteSpaces(frase2)
                    frase2 = frase2.split()
                    for numtoken, token in enumerate(frase2):
                        if devePrintar:
                            print('token:', token)
                            print('numIndiceAnterior:', numIndiceAnterior)
                        if token!='.':
                            numCaracteresTotal = allFrasesString.find(token, numIndiceAnterior, len(allFrasesString))
                        else:
                            numCaracteresTotal = numIndiceAnterior+1
                           
                        #tokens.append([token,numtoken])
                        tokens.append([token,numtoken, numCaracteresTotal])
                        numIndiceAnterior = numCaracteresTotal+len(token)-1
                        if numMaxTokensPorFrase<len(tokens):
                            numMaxTokensPorFrase = len(tokens)
                    frasesTokens[frase]=tokens
                    #linhaTokens=linha.copy()    
                if devePrintar:
                    print('frasesTokens:', frasesTokens)
                # agora, as entidades
                fileAnn = open(f, "r", encoding='utf-8')
                linha=fileAnn.readlines()
                fileAnn.close()

                dicEntidades={}
                for entidade_linha in linha:
                    if ';' not in entidade_linha: # tem descontinua?
                        entidade = entidade_linha.split('\t')
                        tipo_entidade = entidade[1]
                        inicio, fim = tipo_entidade.split()[1:3]
                        tipo_entidade = tipo_entidade.split()[0]
                        termos_entidade = entidade[2].replace('\n','')
                        termos_entidade = termos_entidade.strip().replace('/',' / ').replace(')',' ) ').replace('(',' ( ').replace(']',' ] ').replace('[',' [ ').replace(',',' , ').replace('.',' . ').replace(';',' ; ').replace('-',' - ').replace('+',' + ').replace("'"," ' ").replace(" = "," = ").replace("#"," # ").replace(" $ "," $ ").replace(" ! "," ! ").replace("?"," ? ").replace("%"," % ").replace(":"," : ").replace(">"," > ").replace("<"," < ")
                        dicEntidades[(int(inicio), int(fim))]=[tipo_entidade, replaceWhiteSpaces(termos_entidade)]
                    else:
                        frasesComDescontinuas.append(num)
                        #print('descontinua, linha: {}, num: {}'.format(linha, num))
                        #print('descontinua, file: {}, num: {}'.format(filename, num))
                        entidade = entidade_linha.split('\t')
                        # ex T10	Problema 244 252;279 306	dispneia aos mdoeardos-leves esforço
                        #Problema 244 252;279 306
                        entidade_temp=entidade[1].split(';')
                        entidade1=entidade_temp[0]
                        tipo_entidade = entidade1
                        inicio1, fim1 = tipo_entidade.split()[1:3]
                        tipo_entidade_string = tipo_entidade.split()[0]
                        # mandar só os termos referentes...
                        tamTermo1=int(fim1)-int(inicio1)
                        termos_entidade = entidade[2].replace('\n','')
                        termos_entidade=termos_entidade[:tamTermo1]
                        termos_entidade = termos_entidade.strip().replace('/',' / ').replace(')',' ) ').replace('(',' ( ').replace(']',' ] ').replace('[',' [ ').replace(',',' , ').replace('.',' . ').replace(';',' ; ').replace('-',' - ').replace('+',' + ').replace("'"," ' ").replace(" = "," = ").replace("#"," # ").replace(" $ "," $ ").replace(" ! "," ! ").replace("?"," ? ").replace("%"," % ").replace(":"," : ").replace(">"," > ").replace("<"," < ")
                        #print('aaaaaaaaaaaaaaaaa')
                        dicEntidades[(int(inicio1), int(fim1))]=[tipo_entidade_string, replaceWhiteSpaces(termos_entidade)]
                        #print("(int(inicio1), int(fim1)):", (int(inicio1), int(fim1)))
                        #print("tipo_entidade_string, replaceWhiteSpaces(termos_entidade):", tipo_entidade_string, replaceWhiteSpaces(termos_entidade))
                        
                        entidade2=entidade_temp[1]
                        inicio2, fim2 = entidade2.split()[0:2]
                        termos_entidade = entidade[2].replace('\n','')
                        termos_entidade=termos_entidade[tamTermo1:len(termos_entidade)]
                        termos_entidade = termos_entidade.strip().replace('/',' / ').replace(')',' ) ').replace('(',' ( ').replace(']',' ] ').replace('[',' [ ').replace(',',' , ').replace('.',' . ').replace(';',' ; ').replace('-',' - ').replace('+',' + ').replace("'"," ' ").replace(" = "," = ").replace("#"," # ").replace(" $ "," $ ").replace(" ! "," ! ").replace("?"," ? ").replace("%"," % ").replace(":"," : ").replace(">"," > ").replace("<"," < ")
                        dicEntidades[(int(inicio2), int(fim2))]=[tipo_entidade_string, replaceWhiteSpaces(termos_entidade)]
                        #print("(int(inicio2), int(fim2)):", (int(inicio2), int(fim2)))
                        #print("tipo_entidade_string, replaceWhiteSpaces(termos_entidade):", tipo_entidade_string, replaceWhiteSpaces(termos_entidade))


                #print('frasesTokens:', frasesTokens)
                indicesDic = sorted(dicEntidades.keys(), key = lambda item: item[0])
                listaIndicesJaUsados = []
                #list_students.sort(key = lambda x: x[1])   #index 1 means second element
                #print(indicesDic)
                #print('dicEntidades:', dicEntidades)
                
                for key, value in frasesTokens.items():
                    for i in indicesDic:
                        tipo_entidade,termos_entidade = dicEntidades[i]
                        ##key (i) = indices old a comparar
                        #print('key:', key)
                        #print('value:', value)
                        #print('i:', i)
                        for token in value:
                            #print('token:', token)
                            if i[0]==token[2]:
                                novo_inicio, novo_fim = [token[1],token[1]+len(termos_entidade.split())]
                                novos_indices=[]
                                for k in range(novo_inicio,novo_fim):
                                    novos_indices.append(k)
                                listaEntidades.append([termos_entidade, novos_indices, tipo_entidade])
                                #print("[termos_entidade, novos_indices, tipo_entidade]:", [termos_entidade, novos_indices, tipo_entidade])
                            else:
                                #print('else, i[0]:',i[0])
                                pass
                        
                    if len(value)>0:
                        #print('incluindo:', key)
                        dic_sentences[num]=[value, listaEntidades]
                        listaEntidades=[]
                        num=num+1 

        #print(num)
        #if num>318:
        #    break

        #if num>3:
        #    break
    print('numMaxTokensPorFrase:', numMaxTokensPorFrase)
    return dic_sentences, frasesComDescontinuas
            


In [39]:
dic_sentencesTest, frasesComDescontinuasTest = getDicSentences(pastasCorpus[0])


numMaxTokensPorFrase: 146


In [40]:
dic_sentencesTest[1]

[[['Em', 0, 59],
  ['acompanhamento', 1, 62],
  ['no', 2, 77],
  ['ambualtorio', 3, 80],
  ['há', 4, 92],
  ['5', 5, 95],
  ['anos', 6, 97],
  ['por', 7, 102],
  ['FA', 8, 106],
  [',', 9, 108],
  ['uso', 10, 110],
  ['de', 11, 114],
  ['marevan', 12, 117],
  ['5mg', 13, 125],
  ['1', 14, 129],
  ['x', 15, 131],
  ['ao', 16, 133],
  ['dia', 17, 136],
  ['.', 18, 139]],
 [['FA', [8], 'Problema'], ['marevan 5mg', [12, 13], 'Tratamento']]]

In [41]:
dic_sentencesTrain, frasesComDescontinuasTrain = getDicSentences(pastasCorpus[1])


numMaxTokensPorFrase: 192


In [42]:
print('--Treinamento:--', pastasCorpus[1])
dic_sentencesTrain, frasesComDescontinuasTrain = getDicSentences(pastasCorpus[1])
print('len(sentences):', len(dic_sentencesTrain))
print('len(Descontinuas):',len(frasesComDescontinuasTrain))
print('len(frasesComDescontinuas):',len(set(frasesComDescontinuasTrain)))


print('--Teste:--', pastasCorpus[0])
dic_sentencesTest, frasesComDescontinuasTest = getDicSentences(pastasCorpus[0])
print('len(sentences):', len(dic_sentencesTest))
print('len(Descontinuas):',len(frasesComDescontinuasTest))
print('len(frasesComDescontinuas):',len(set(frasesComDescontinuasTest)))


--Treinamento:-- corpus\train
numMaxTokensPorFrase: 192
len(sentences): 1736
len(Descontinuas): 91
len(frasesComDescontinuas): 51
--Teste:-- corpus\test
numMaxTokensPorFrase: 146
len(sentences): 506
len(Descontinuas): 44
len(frasesComDescontinuas): 17


In [43]:
dic_sentencesTest[9]

[[['Abd', 0, 644],
  ['globoso', 1, 648],
  [',', 2, 655],
  ['flacido', 3, 657],
  [',', 4, 664],
  ['indolor', 5, 666],
  ['a', 6, 674],
  ['palpacao', 7, 676],
  [',', 8, 684],
  ['sem', 9, 686],
  ['VCM', 10, 690],
  ['.', 11, 693]],
 [['Abd', [0], 'Anatomia'], ['VCM', [10], 'Problema']]]

In [44]:
dic_sentencesTrain[861]

[[['CONTRAÇÃO', 0, 507],
  ['SEGMENTAR', 1, 517],
  ['DO', 2, 527],
  ['VE', 3, 530],
  ['ALTERADA', 4, 533],
  ['.', 5, 541]],
 [['CONTRAÇÃO SEGMENTAR DO VE ALTERADA', [0, 1, 2, 3, 4], 'Problema'],
  ['VE', [3], 'Anatomia']]]

In [45]:
frasesComDescontinuasTrain[:10]

[3, 15, 30, 30, 30, 30, 55, 55, 65, 65]

In [46]:
# frases com entidades descontinuas
print(len(set(frasesComDescontinuasTrain)))

51


In [47]:
frasesComDescontinuasTest

[0,
 42,
 42,
 42,
 49,
 67,
 67,
 67,
 67,
 67,
 91,
 131,
 147,
 197,
 197,
 197,
 197,
 197,
 197,
 197,
 197,
 197,
 197,
 197,
 227,
 241,
 241,
 241,
 241,
 241,
 241,
 241,
 263,
 291,
 291,
 291,
 316,
 329,
 367,
 367,
 367,
 367,
 406,
 468]

In [48]:
print(dic_sentencesTest[2])

[[['Comorbidades', 0, 142], [':', 1, 154], ['DM', 2, 156], ['há', 3, 159], ['10', 4, 162], ['anos', 5, 165], ['em', 6, 170], ['uso', 7, 173], ['de', 8, 177], ['metformina', 9, 180], ['850mg', 10, 191], ['3', 11, 197], ['cp', 12, 199], ['/', 13, 201], ['dia', 14, 202], [',', 15, 205], ['acarbose', 16, 207], ['1', 17, 216], ['cp', 18, 218], ['/', 19, 220], ['dia', 20, 221], ['e', 21, 225], ['glicazida', 22, 227], ['60mg', 23, 237], ['2', 24, 242], ['cp', 25, 244], ['/', 26, 246], ['dia', 27, 247], ['e', 28, 251], ['insulina', 29, 253], ['(', 30, 262], ['24', 31, 263], ['-', 32, 266], ['0', 33, 268], ['-', 34, 270], ['24', 35, 272], [')', 36, 274], ['.', 37, 275]], [['Comorbidades', [0], 'Problema'], ['DM', [2], 'Problema'], ['metformina 850mg', [9, 10], 'Tratamento'], ['acarbose', [16], 'Tratamento'], ['glicazida 60mg', [22, 23], 'Tratamento'], ['insulina', [29], 'Tratamento']]]


In [49]:
save_obj('dic_sentencesTrainDev',dic_sentencesTrain)
porc=0.76
tamanhoTotal = len(dic_sentencesTrain)
tamanhoTrain = int(tamanhoTotal*porc)
print('tamanhoTrain', tamanhoTrain)
tamanhoDev = tamanhoTotal - tamanhoTrain
print('tamanhoDev', tamanhoDev)
dic_sentencesDev_temp = {k: dic_sentencesTrain[k] for k in list(dic_sentencesTrain)[tamanhoTrain:-1]}
dic_sentencesTrain = {k: dic_sentencesTrain[k] for k in list(dic_sentencesTrain)[:tamanhoTrain]}
num=0
dic_sentencesDev = {}
for key, value in dic_sentencesDev_temp.items():
    dic_sentencesDev[num] = value
    num=num+1

print('len(dic_sentencesTrain):', len(dic_sentencesTrain))
print('len(dic_sentencesDev):', len(dic_sentencesDev))
print(dic_sentencesTrain[tamanhoTrain-1])
print(dic_sentencesDev[0])

tamanhoTrain 1319
tamanhoDev 417
len(dic_sentencesTrain): 1319
len(dic_sentencesDev): 416
[[['Paciente', 0, 151], ['relata', 1, 160], ['apenas', 2, 167], ['um', 3, 174], ['episodio', 4, 177], ['no', 5, 186], ['momento', 6, 189], ['de', 7, 197], ['gripe', 8, 200], ['.', 9, 205]], [['gripe', [8], 'Problema']]]
[[['HAS', 0, 207], [',', 1, 210], ['ICC', 2, 212], [',', 3, 215], ['nega', 4, 217], ['DM', 5, 222], ['.', 6, 224]], [['HAS', [0], 'Problema'], ['ICC', [2], 'Problema'], ['DM', [5], 'Problema']]]


In [50]:
dic_sentencesDev[0]

[[['HAS', 0, 207],
  [',', 1, 210],
  ['ICC', 2, 212],
  [',', 3, 215],
  ['nega', 4, 217],
  ['DM', 5, 222],
  ['.', 6, 224]],
 [['HAS', [0], 'Problema'], ['ICC', [2], 'Problema'], ['DM', [5], 'Problema']]]

In [51]:
a = [i[2] for i in dic_sentencesDev[1][0] if i[1]==16]
a

[280]

In [52]:
[len(i[0]) for i in dic_sentencesDev[1][0] if i[1]==1][0]

1

In [53]:
texto='123'
texto[:len(texto)-1]

'12'

In [54]:
dic_sentencesTest[1]

[[['Em', 0, 59],
  ['acompanhamento', 1, 62],
  ['no', 2, 77],
  ['ambualtorio', 3, 80],
  ['há', 4, 92],
  ['5', 5, 95],
  ['anos', 6, 97],
  ['por', 7, 102],
  ['FA', 8, 106],
  [',', 9, 108],
  ['uso', 10, 110],
  ['de', 11, 114],
  ['marevan', 12, 117],
  ['5mg', 13, 125],
  ['1', 14, 129],
  ['x', 15, 131],
  ['ao', 16, 133],
  ['dia', 17, 136],
  ['.', 18, 139]],
 [['FA', [8], 'Problema'], ['marevan 5mg', [12, 13], 'Tratamento']]]

In [55]:
'''
def corrigeIndices(dic_sentencesTest):
    dic_sentencesTest2 = {}
    for key, value in dic_sentencesTest.items():
        tokens =value[0] 
        tokens2=tokens.copy()
        valorDesconto=0
        listaTokens = []
        for i, token in enumerate(tokens2):
            #print(token)
            ultimoIndice=token[2]
            if i==0:
                valorDesconto = ultimoIndice
            ultimoIndiceCorrigido = ultimoIndice - valorDesconto
            listaTokens.append([token[0], token[1], ultimoIndiceCorrigido])
            #print(ultimoIndiceCorrigido)
        #print(tokens2)
        dic_sentencesTest2[key] = [listaTokens, value[1]]
    return dic_sentencesTest2

dic_sentencesTest2 = corrigeIndices(dic_sentencesTest)
dic_sentencesTest2[3]
'''

'\ndef corrigeIndices(dic_sentencesTest):\n    dic_sentencesTest2 = {}\n    for key, value in dic_sentencesTest.items():\n        tokens =value[0] \n        tokens2=tokens.copy()\n        valorDesconto=0\n        listaTokens = []\n        for i, token in enumerate(tokens2):\n            #print(token)\n            ultimoIndice=token[2]\n            if i==0:\n                valorDesconto = ultimoIndice\n            ultimoIndiceCorrigido = ultimoIndice - valorDesconto\n            listaTokens.append([token[0], token[1], ultimoIndiceCorrigido])\n            #print(ultimoIndiceCorrigido)\n        #print(tokens2)\n        dic_sentencesTest2[key] = [listaTokens, value[1]]\n    return dic_sentencesTest2\n\ndic_sentencesTest2 = corrigeIndices(dic_sentencesTest)\ndic_sentencesTest2[3]\n'

In [56]:
def corrigeIndices(dic_sentencesTest):
    dic_sentencesTest2 = {}
    for key, value in dic_sentencesTest.items():
        tokens =value[0] 
        tokens2=tokens.copy()
        listaTokens = []
        indiceAnterior=0
        tamanhoAnterior=0
        for i, token in enumerate(tokens2):
            #print(token)
            if i==0:
                indice=0
            else:
                indice = indiceAnterior+tamanhoAnterior+1
            #indice=token[2]
            listaTokens.append([token[0], token[1], indice])
            indiceAnterior = indice
            tamanhoAnterior = len(token[0])
        dic_sentencesTest2[key]=[listaTokens, value[1]]
    return dic_sentencesTest2

dic_sentencesTest2 = corrigeIndices(dic_sentencesTest)
dic_sentencesTest2[3]

[[['HAS', 0, 0],
  ['há', 1, 4],
  ['15', 2, 7],
  ['anos', 3, 10],
  ['em', 4, 15],
  ['uso', 5, 18],
  ['de', 6, 22],
  ['losartana', 7, 25],
  ['50mg', 8, 35],
  ['/', 9, 40],
  ['dia', 10, 42],
  ['e', 11, 46],
  ['digoxina', 12, 48],
  ['1', 13, 57],
  ['/', 14, 59],
  ['2', 15, 61],
  ['cp', 16, 63],
  ['/', 17, 66],
  ['dia', 18, 68],
  [',', 19, 72],
  ['carvedilol', 20, 74],
  ['25', 21, 85],
  ['12', 22, 88],
  ['/', 23, 91],
  ['12', 24, 93],
  [',', 25, 96],
  ['HCTZ', 26, 98],
  ['.', 27, 103]],
 [['HAS', [0], 'Problema'],
  ['losartana 50mg', [7, 8], 'Tratamento'],
  ['digoxina', [12], 'Tratamento'],
  ['carvedilol 25', [20, 21], 'Tratamento'],
  ['HCTZ', [26], 'Tratamento']]]

In [57]:
dic_sentencesTest2[2]

[[['Comorbidades', 0, 0],
  [':', 1, 13],
  ['DM', 2, 15],
  ['há', 3, 18],
  ['10', 4, 21],
  ['anos', 5, 24],
  ['em', 6, 29],
  ['uso', 7, 32],
  ['de', 8, 36],
  ['metformina', 9, 39],
  ['850mg', 10, 50],
  ['3', 11, 56],
  ['cp', 12, 58],
  ['/', 13, 61],
  ['dia', 14, 63],
  [',', 15, 67],
  ['acarbose', 16, 69],
  ['1', 17, 78],
  ['cp', 18, 80],
  ['/', 19, 83],
  ['dia', 20, 85],
  ['e', 21, 89],
  ['glicazida', 22, 91],
  ['60mg', 23, 101],
  ['2', 24, 106],
  ['cp', 25, 108],
  ['/', 26, 111],
  ['dia', 27, 113],
  ['e', 28, 117],
  ['insulina', 29, 119],
  ['(', 30, 128],
  ['24', 31, 130],
  ['-', 32, 133],
  ['0', 33, 135],
  ['-', 34, 137],
  ['24', 35, 139],
  [')', 36, 142],
  ['.', 37, 144]],
 [['Comorbidades', [0], 'Problema'],
  ['DM', [2], 'Problema'],
  ['metformina 850mg', [9, 10], 'Tratamento'],
  ['acarbose', [16], 'Tratamento'],
  ['glicazida 60mg', [22, 23], 'Tratamento'],
  ['insulina', [29], 'Tratamento']]]

In [58]:
dic_sentencesTrain2 = corrigeIndices(dic_sentencesTrain)
dic_sentencesTest2 = corrigeIndices(dic_sentencesTest)
dic_sentencesDev2 = corrigeIndices(dic_sentencesDev)


In [59]:
def gravaArquivoQA(dic_sentencesTest, name):
    tipos= ['Problema','Teste','Tratamento','Anatomia']

    texto=r'{"data": ['
    # title
    # paragraphs
        #context, qas (answers (answer_start_original, text_original)), question (id, question_original)
    num=0
    idQa=1000000

    for i in range(len(dic_sentencesTest)):
        ents = dic_sentencesTest[i][1]

        listaEntidadesProblema=list()
        listaEntidadesTeste=list()
        listaEntidadesTratamento=list()
        listaEntidadesAnatomia=list()


        #print(contexto)
        for entidade in ents:
            if entidade[2] == 'Problema':
                listaEntidadesProblema.append(entidade)
            elif entidade[2] == 'Teste':
                listaEntidadesTeste.append(entidade)
            elif entidade[2] == 'Tratamento':
                listaEntidadesTratamento.append(entidade)
            elif entidade[2] == 'Anatomia':
                listaEntidadesAnatomia.append(entidade)

        if len(listaEntidadesProblema)==0 and len(listaEntidadesTeste)==0 and len(listaEntidadesTratamento)==0 and len(listaEntidadesAnatomia)==0:
            continue

        title='texto_'+str(i)
        if num==0:
            texto = texto+'{"title":"'+title+'","paragraphs": [{"context": "'
        else:
            texto = texto[:len(texto)-1]
            texto = texto+']}]}, {"title":"'+title+'","paragraphs": [{"context": "'
        
        num=num+1

        tokens = dic_sentencesTest[i][0]
        frase=[t[0] for t in tokens]
        contexto = ' '.join(frase)
        #contexto = contexto.replace(' , ', ', ').replace(' : ', ': ').replace(' . ', '. ').replace(' / ', '/ ')
        if '"' in contexto:
            print('tem aspassssss')
        texto = texto+contexto+'", "qas": ['


        if len(listaEntidadesProblema)>0:
            if '"' in entidade[0]:
                print('tem aspassssss22')
            idQa=idQa+1
            texto = texto+'{"answers": ['
            for entidade in listaEntidadesProblema:
                texto = texto+'{"answer_start":'
                indices = entidade[1]
                start =indices[0]
                fim = indices[-1]
                start_string = [i[2] for i in dic_sentencesTest[i][0] if i[1]==start][0]
                fim_string = [i[2] for i in dic_sentencesTest[i][0] if i[1]==fim][0]
                fim_string = fim_string+[len(i[0]) for i in dic_sentencesTest[i][0] if i[1]==fim][0]

                texto = texto+str(start_string)+', "text": "'+entidade[0]+'"},'
                

            texto = texto[:len(texto)-1]
            texto = texto+'], "question": "Problema", "id":"'+str(idQa)+'"},'

        if len(listaEntidadesTeste)>0:
            if '"' in entidade[0]:
                print('tem aspassssss22')
            idQa=idQa+1
            texto = texto+'{"answers": ['
            for entidade in listaEntidadesTeste:
                texto = texto+'{"answer_start":'
                indices = entidade[1]
                start =indices[0]
                fim = indices[-1]
                try:
                    start_string = [j[2] for j in dic_sentencesTest[i][0] if j[1]==start][0]
                except:
                    print()
                    raise
                fim_string = [j[2] for j in dic_sentencesTest[i][0] if j[1]==fim][0]
                fim_string = fim_string+[len(j[0]) for j in dic_sentencesTest[i][0] if j[1]==fim][0]

                texto = texto+str(start_string)+', "text": "'+entidade[0]+'"},'

            texto = texto[:len(texto)-1]
            texto = texto+'], "question": "Teste", "id":"'+str(idQa)+'"},'

        if len(listaEntidadesTratamento)>0:
            if '"' in entidade[0]:
                print('tem aspassssss22')
            idQa=idQa+1
            texto = texto+'{"answers": ['
            for entidade in listaEntidadesTratamento:
                texto = texto+'{"answer_start":'
                indices = entidade[1]
                start =indices[0]
                fim = indices[-1]
                start_string = [i[2] for i in dic_sentencesTest[i][0] if i[1]==start][0]
                fim_string = [i[2] for i in dic_sentencesTest[i][0] if i[1]==fim][0]
                fim_string = fim_string+[len(i[0]) for i in dic_sentencesTest[i][0] if i[1]==fim][0]

                texto = texto+str(start_string)+', "text": "'+entidade[0]+'"},'

            texto = texto[:len(texto)-1]
            texto = texto+'], "question": "Tratamento", "id":"'+str(idQa)+'"},'

        if len(listaEntidadesAnatomia)>0:
            if '"' in entidade[0]:
                print('tem aspassssss22')
            idQa=idQa+1
            texto = texto+'{"answers": ['
            for entidade in listaEntidadesAnatomia:
                texto = texto+'{"answer_start":'
                indices = entidade[1]
                start =indices[0]
                fim = indices[-1]
                start_string = [i[2] for i in dic_sentencesTest[i][0] if i[1]==start][0]
                fim_string = [i[2] for i in dic_sentencesTest[i][0] if i[1]==fim][0]
                fim_string = fim_string+[len(i[0]) for i in dic_sentencesTest[i][0] if i[1]==fim][0]

                texto = texto+str(start_string)+', "text": "'+entidade[0]+'"},'

            # tirando ultima virgula
            texto = texto[:len(texto)-1]
            texto = texto+'], "question": "Anatomia", "id":"'+str(idQa)+'"},'

    texto = texto[:len(texto)-1]
    texto = texto+']}]}], "version": "1.1"}'   
    arq = open(name, 'w', encoding='utf8')
    arq.write(texto)
    arq.close()
    #print(texto)
  
gravaArquivoQA(dic_sentencesTest2, 'data_qa/nested_test.json')
gravaArquivoQA(dic_sentencesTrain2, 'data_qa/nested_train.json')
gravaArquivoQA(dic_sentencesDev2, 'data_qa/nested_dev.json')

In [161]:
import json 

try:
  file = r'data_qa/nested_test.json'
  # Opening JSON file & returns JSON object as a dictionary 
  #f = open(file, encoding="utf8") 
  f = open(file, encoding="utf8") 
  data = json.load(f) 
except:
  print('nao abriu arquivo')
  raise

In [31]:
dic_sentencesTrain2[0]

[[['Dispneia', 0, 0],
  ['importante', 1, 9],
  ['aos', 2, 20],
  ['esforços', 3, 24],
  ['+', 4, 33],
  ['dor', 5, 35],
  ['tipo', 6, 39],
  ['peso', 7, 44],
  ['no', 8, 49],
  ['peito', 9, 52],
  ['no', 10, 58],
  ['esforço', 11, 61],
  ['.', 12, 69]],
 [['Dispneia importante aos esforços', [0, 1, 2, 3], 'Problema'],
  ['dor tipo peso no peito no esforço', [5, 6, 7, 8, 9, 10, 11], 'Problema'],
  ['peito', [9], 'Anatomia']]]

# squad v2

In [34]:
# versão 2 do squad
def gravaArquivoQA_impossible(dic_sentences, name):
    print('name:', name)
    tipos= ['Problema','Teste','Tratamento','Anatomia']

    texto=r'{"version": "v2.0", "data": ['
    # title
    # paragraphs
        #context, qas (answers (answer_start_original, text_original)), question (id, question_original)
    num=0
    idQa=1000000

    for i in range(len(dic_sentences)):
        ents = dic_sentences[i][1]

        listaEntidadesProblema=list()
        listaEntidadesTeste=list()
        listaEntidadesTratamento=list()
        listaEntidadesAnatomia=list()

        #print(contexto)
        for entidade in ents:
            if entidade[2] == 'Problema':#problema
                listaEntidadesProblema.append(entidade)
            elif entidade[2] == 'Teste': #teste
                listaEntidadesTeste.append(entidade)
            elif entidade[2] == 'Tratamento':#tratamento
                listaEntidadesTratamento.append(entidade)
            elif entidade[2] == 'Anatomia':#anatomia
                listaEntidadesAnatomia.append(entidade)

        if len(listaEntidadesProblema)==0 and len(listaEntidadesTeste)==0 and len(listaEntidadesTratamento)==0 and len(listaEntidadesAnatomia)==0:
            continue

        title='texto_'+str(i)
        if num==0:
            texto = texto+'{"title":"'+title+'","paragraphs": [{"qas": ['
        else:
            texto = texto[:len(texto)-1]
            texto = texto+'}]}, {"title":"'+title+'","paragraphs": [{"qas": ['
        
        num=num+1

        tokens = dic_sentences[i][0]
        frase=[t[0] for t in tokens]
        contexto = ' '.join(frase)
        
        
        if len(listaEntidadesProblema)>0:
            if '"' in entidade[0]:
                print('tem aspas em Problema')
            idQa=idQa+1
            texto = texto+'{"question": "Problema", "id":"'+str(idQa)+'", "answers": ['
            
            for entidade in listaEntidadesProblema:
                #print(dic_sentencesTest[i][0])
                texto = texto+'{"text":"'+entidade[0]+'", "answer_start":'
                indices = entidade[1]
                start =indices[0]
                fim = indices[-1]
                start_string = [i[2] for i in dic_sentences[i][0] if i[1]==start][0]
                fim_string = [i[2] for i in dic_sentences[i][0] if i[1]==fim][0]
                fim_string = fim_string+[len(i[0]) for i in dic_sentences[i][0] if i[1]==fim][0]

                texto = texto+str(start_string)+'},'
                #"is_impossible": false
                
            texto = texto[:len(texto)-1]
            texto = texto + '], "is_impossible": false},'
        
        else:
            idQa=idQa+1
            texto = texto+'{"question": "Problema", "id":"'+str(idQa)+'", "answers": [], "is_impossible": true},'

        if len(listaEntidadesTeste)>0:
            if '"' in entidade[0]:
                print('tem aspas em Teste')
            idQa=idQa+1
            texto = texto+'{"question": "Teste", "id":"'+str(idQa)+'", "answers": ['
            
            for entidade in listaEntidadesTeste:
                #print(dic_sentencesTest[i][0])
                texto = texto+'{"text":"'+entidade[0]+'", "answer_start":'
                indices = entidade[1]
                start =indices[0]
                fim = indices[-1]
                start_string = [i[2] for i in dic_sentences[i][0] if i[1]==start][0]
                fim_string = [i[2] for i in dic_sentences[i][0] if i[1]==fim][0]
                fim_string = fim_string+[len(i[0]) for i in dic_sentences[i][0] if i[1]==fim][0]

                texto = texto+str(start_string)+'},'
                #"is_impossible": false
                
            texto = texto[:len(texto)-1]
            texto = texto + '], "is_impossible": false},'
        
        else:
            idQa=idQa+1
            texto = texto+'{"question": "Teste", "id":"'+str(idQa)+'", "answers": [], "is_impossible": true},'

        if len(listaEntidadesTratamento)>0:
            if '"' in entidade[0]:
                print('tem aspas Tratamento')
            idQa=idQa+1
            texto = texto+'{"question": "Tratamento", "id":"'+str(idQa)+'", "answers": ['
            
            for entidade in listaEntidadesTratamento:
                #print(dic_sentencesTest[i][0])
                texto = texto+'{"text":"'+entidade[0]+'", "answer_start":'
                indices = entidade[1]
                start =indices[0]
                fim = indices[-1]
                start_string = [i[2] for i in dic_sentences[i][0] if i[1]==start][0]
                fim_string = [i[2] for i in dic_sentences[i][0] if i[1]==fim][0]
                fim_string = fim_string+[len(i[0]) for i in dic_sentences[i][0] if i[1]==fim][0]

                texto = texto+str(start_string)+'},'
                #"is_impossible": false
                
            texto = texto[:len(texto)-1]
            texto = texto + '], "is_impossible": false},'
        
        else:
            idQa=idQa+1
            texto = texto+'{"question": "Tratamento", "id":"'+str(idQa)+'", "answers": [], "is_impossible": true},'

        if len(listaEntidadesAnatomia)>0:
            if '"' in entidade[0]:
                print('tem aspas em Anatomia')
            idQa=idQa+1
            texto = texto+'{"question": "Anatomia", "id":"'+str(idQa)+'", "answers": ['
            
            for entidade in listaEntidadesAnatomia:
                #print(dic_sentencesTest[i][0])
                texto = texto+'{"text":"'+entidade[0]+'", "answer_start":'
                indices = entidade[1]
                start =indices[0]
                fim = indices[-1]
                start_string = [i[2] for i in dic_sentences[i][0] if i[1]==start][0]
                fim_string = [i[2] for i in dic_sentences[i][0] if i[1]==fim][0]
                fim_string = fim_string+[len(i[0]) for i in dic_sentences[i][0] if i[1]==fim][0]

                texto = texto+str(start_string)+'},'
                #"is_impossible": false
                
            texto = texto[:len(texto)-1]
            texto = texto + '], "is_impossible": false},'
        
        else:
            idQa=idQa+1
            texto = texto+'{"question": "Anatomia", "id":"'+str(idQa)+'", "answers": [], "is_impossible": true},'   
                    
        texto = texto[:len(texto)-1]
        texto = texto+'], "context":"'+contexto+'"}' 
        
    texto = texto+']}]}'

    arq = open(name, 'w', encoding='utf8')
    arq.write(texto)
    arq.close()
    #print(texto)
  
gravaArquivoQA_impossible(dic_sentencesTest2, 'data_qa_impossible/nested_test_impossible.json')
gravaArquivoQA_impossible(dic_sentencesTrain2, 'data_qa_impossible/nested_train_impossible.json')
gravaArquivoQA_impossible(dic_sentencesDev2, 'data_qa_impossible/nested_dev_impossible.json')

name: data_qa_impossible/nested_test_impossible.json
name: data_qa_impossible/nested_train_impossible.json
name: data_qa_impossible/nested_dev_impossible.json


# gerando agora no formato do MRC

In [63]:
# versão MRC

def gravaArquivoQA_mrc(dic_sentences, name):
    print('name:', name)
    tipos= ['Problema','Teste','Tratamento','Anatomia']

    texto=r'['
    # title
    # paragraphs
        #context, qas (answers (answer_start_original, text_original)), question (id, question_original)
    num=0
    idQa=1000000

    for i in range(len(dic_sentences)):
        ents = dic_sentences[i][1]

        listaEntidadesProblema=list()
        listaEntidadesTeste=list()
        listaEntidadesTratamento=list()
        listaEntidadesAnatomia=list()

        #print(contexto)
        for entidade in ents:
            if entidade[2] == 'Problema':#problema
                listaEntidadesProblema.append(entidade)
            elif entidade[2] == 'Teste': #teste
                listaEntidadesTeste.append(entidade)
            elif entidade[2] == 'Tratamento':#tratamento
                listaEntidadesTratamento.append(entidade)
            elif entidade[2] == 'Anatomia':#anatomia
                listaEntidadesAnatomia.append(entidade)

        if len(listaEntidadesProblema)==0 and len(listaEntidadesTeste)==0 and len(listaEntidadesTratamento)==0 and len(listaEntidadesAnatomia)==0:
            continue

        tokens = dic_sentences[i][0]
        frase=[t[0] for t in tokens]
        contexto = ' '.join(frase)
        
        texto = texto+'{"context":"'+contexto+'",' 
        
        #if len(listaEntidadesProblema)>0:
        tipo='Problema'
        if '"' in entidade[0]:
            print('tem aspas em ', tipo)
        idQa=idQa+1

        ends=[]
        starts=[]

        for entidade in listaEntidadesProblema:
            indices = entidade[1]
            start =indices[0]
            fim = indices[-1]
            #start_string = [i[2] for i in dic_sentences[i][0] if i[1]==start][0]
            #fim_string = [i[2] for i in dic_sentences[i][0] if i[1]==fim][0]
            #fim_string = fim_string+[len(i[0]) for i in dic_sentences[i][0] if i[1]==fim][0]
            #starts = [6,13]
            starts.append(start)
            ends.append(fim)

        spans_pos=''
        impossible="true"
        spans_pos = '[]'
        if len(starts)>0:
            impossible="false"
            spans_pos = '['
            for a, b in zip(starts, ends):
                spans_pos = spans_pos +'"'+ str(a)+';'+str(b)+'",'
            spans_pos =spans_pos[0:-1]+ ']'


        texto = texto+'"end_position": '+str(ends)+', "entity_label":"'+tipo+'", "impossible":'+impossible+', "qas_id": "'+ str(idQa)+'.1",'
        texto = texto+'"query":"'+tipo+'", "span_position":'+str(spans_pos)+', "start_position":'+str(starts)+"}," 

        texto = texto+'{"context":"'+contexto+'",' 
        
        tipo='Teste'
        if '"' in entidade[0]:
            print('tem aspas em ', tipo)
        idQa=idQa+1

        ends=[]
        starts=[]

        for entidade in listaEntidadesTeste:
            indices = entidade[1]
            start =indices[0]
            fim = indices[-1]
            starts.append(start)
            ends.append(fim)

        spans_pos=''
        impossible="true"
        spans_pos = '[]'
        if len(starts)>0:
            impossible="false"
            spans_pos = '['
            for a, b in zip(starts, ends):
                spans_pos = spans_pos +'"'+ str(a)+';'+str(b)+'",'
            spans_pos =spans_pos[0:-1]+ ']'


        texto = texto+'"end_position": '+str(ends)+', "entity_label":"'+tipo+'", "impossible":'+impossible+', "qas_id": "'+ str(idQa)+'.1",'
        texto = texto+'"query":"'+tipo+'", "span_position":'+str(spans_pos)+', "start_position":'+str(starts)+"}," 

        texto = texto+'{"context":"'+contexto+'",' 
        tipo='Tratamento'
        if '"' in entidade[0]:
            print('tem aspas em ', tipo)
        idQa=idQa+1

        ends=[]
        starts=[]

        for entidade in listaEntidadesTratamento:
            indices = entidade[1]
            start =indices[0]
            fim = indices[-1]
            starts.append(start)
            ends.append(fim)

        spans_pos=''
        impossible="true"
        spans_pos = '[]'
        if len(starts)>0:
            impossible="false"
            spans_pos = '['
            for a, b in zip(starts, ends):
                spans_pos = spans_pos +'"'+ str(a)+';'+str(b)+'",'
            spans_pos =spans_pos[0:-1]+ ']'


        texto = texto+'"end_position": '+str(ends)+', "entity_label":"'+tipo+'", "impossible":'+impossible+', "qas_id": "'+ str(idQa)+'.1",'
        texto = texto+'"query":"'+tipo+'", "span_position":'+str(spans_pos)+', "start_position":'+str(starts)+"}," 

        texto = texto+'{"context":"'+contexto+'",' 
        tipo='Anatomia'
        if '"' in entidade[0]:
            print('tem aspas em ', tipo)
        idQa=idQa+1

        ends=[]
        starts=[]

        for entidade in listaEntidadesAnatomia:
            indices = entidade[1]
            start =indices[0]
            fim = indices[-1]
            starts.append(start)
            ends.append(fim)

        spans_pos=''
        impossible="true"
        spans_pos = '[]'
        if len(starts)>0:
            impossible="false"
            spans_pos = '['
            for a, b in zip(starts, ends):
                spans_pos = spans_pos +'"'+ str(a)+';'+str(b)+'",'
            spans_pos =spans_pos[0:-1]+ ']'


        texto = texto+'"end_position": '+str(ends)+', "entity_label":"'+tipo+'", "impossible":'+impossible+', "qas_id": "'+ str(idQa)+'.1",'
        texto = texto+'"query":"'+tipo+'", "span_position":'+str(spans_pos)+', "start_position":'+str(starts)+"}," 

        
    #texto = texto+']'
    texto =texto[0:-1]+ ']'

    arq = open(name, 'w', encoding='utf8')
    arq.write(texto)
    arq.close()
    #print(texto)
  
gravaArquivoQA_mrc(dic_sentencesTest2, 'data_qa_mrc/nested/mrc-ner.test')
gravaArquivoQA_mrc(dic_sentencesTrain2, 'data_qa_mrc/nested/mrc-ner.train')
gravaArquivoQA_mrc(dic_sentencesDev2, 'data_qa_mrc/nested/mrc-ner.dev')

name: data_qa_mrc/nested/mrc-ner.test


In [62]:
dic_sentencesDev2[0]

[[['HAS', 0, 0],
  [',', 1, 4],
  ['ICC', 2, 6],
  [',', 3, 10],
  ['nega', 4, 12],
  ['DM', 5, 17],
  ['.', 6, 20]],
 [['HAS', [0], 'Problema'], ['ICC', [2], 'Problema'], ['DM', [5], 'Problema']]]

In [55]:
#starts = [2,3,4]
#ends=[13,14,15]
starts = []
ends=[]
#spans_pos = [str(a)+";"+str(b) for a, b in zip(starts, ends)]
spans_pos = '['
for a, b in zip(starts, ends):
    spans_pos = spans_pos +'"'+ str(a)+';'+str(b)+'",'
spans_pos =spans_pos[0:-1]+ ']'
spans_pos
#print(str(starts))

']'